<a href="https://colab.research.google.com/github/Dineshkumar128/ISYS2001-S1-2025/blob/main/Week%207%20Notebooks/activity_2_data_analysis_with_pandas_student_walkthrough.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Guided Data Analysis with Pandas
**A Step-by-Step Walkthrough with Explanations**

## Learning Objectives
- Learn to load and inspect real-world data using pandas.
- Understand and clean missing or extreme values using guided examples.
- Build skills in reasoning about data quality and cleaning logic.


## Introduction

In this worksheet, you’ll walk through a realistic data analysis task.  
Each step is explained and broken into parts to help you understand *what*, *how*, and *why* we do it.

We’ll work with a sample sales dataset and explore:
- Loading and previewing data
- Finding and removing missing values
- Detecting and filtering outliers using the IQR method

You’ll be encouraged to stop, observe output, and reflect along the way!


In [3]:
import pandas as pd

## Step 1: Load the Data

Let’s start by loading the sales dataset from a CSV file.  
Replace `'sales_data.csv'` with your file name if needed.


In [4]:
# Load dataset
df = pd.read_csv('sales_data.csv')

# Print the first few rows
df.head()

,date,store_id,product_id,units_sold,unit_price,sales
0,2024-11-03,1001,262,23.0,29.55,679.65
1,2022-05-14,1005,256,20.0,25.32,506.40
2,2023-11-07,1008,245,25.0,31.58,789.50
3,2024-08-20,1000,232,19.0,25.91,492.29
4,2023-03-05,1001,255,28.0,22.45,628.60


### 🔍 Checkpoint:
- What columns do you see?
- What does each row represent?


## Step 2: Inspect the DataFrame

Check the overall structure using `.info()` and `.describe()`.  
This tells you about:
- Missing values
- Column data types
- Summary statistics (like average, min, max)


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        10000 non-null  object 
 1   store_id    10000 non-null  int64  
 2   product_id  10000 non-null  int64  
 3   units_sold  9700 non-null   float64
 4   unit_price  9700 non-null   float64
 5   sales       10000 non-null  float64
dtypes: float64(3), int64(2), object(1)
memory usage: 468.9+ KB


In [6]:
df.describe()

,store_id,product_id,units_sold,unit_price,sales
count,10000.000000,10000.000000,9700.000000,9700.000000,10000.000000
mean,1004.455800,249.183200,19.982784,25.078123,544.512962
std,2.899877,28.972314,4.472414,4.951588,466.388013
min,1000.000000,200.000000,6.000000,6.090000,103.180000
25%,1002.000000,224.000000,17.000000,21.740000,393.582500
50%,1004.000000,249.000000,20.000000,25.100000,492.015000
75%,1007.000000,274.000000,23.000000,28.450000,600.865000
max,1009.000000,299.000000,38.000000,46.010000,7763.300000


### 🧠 Think About:
- Are any columns missing lots of data?
- Do any values look unusually large or small?


## Step 3: Remove Missing Values

Missing values can mess up calculations or visualisations.  
Let’s remove any rows with missing values using `.dropna()`.


In [7]:
# Drop missing rows
df_clean = df.dropna()

# Compare size before and after
print("Original rows:", len(df))
print("Cleaned rows:", len(df_clean))

# Preview cleaned data
df_clean.head()

Original rows: 10000
Cleaned rows: 9410


,date,store_id,product_id,units_sold,unit_price,sales
0,2024-11-03,1001,262,23.0,29.55,679.65
1,2022-05-14,1005,256,20.0,25.32,506.40
2,2023-11-07,1008,245,25.0,31.58,789.50
3,2024-08-20,1000,232,19.0,25.91,492.29
4,2023-03-05,1001,255,28.0,22.45,628.60


### 🧠 Why Do This?
If you don’t remove or handle missing values, some functions (like `.mean()`) may not work properly.


## Step 4: Remove Outliers Using IQR

Outliers are extreme values that can distort your analysis.  
We’ll use a common method called the **IQR Rule**:

1. Find Q1 (25th percentile) and Q3 (75th percentile).
2. Calculate IQR = Q3 - Q1.
3. Define a valid range:
    - Lower bound = Q1 - 1.5 × IQR  
    - Upper bound = Q3 + 1.5 × IQR
4. Keep only rows within that range.


In [9]:
# Step-by-step: calculate bounds
Q1 = df_clean['sales'].quantile(0.25)
Q3 = df_clean['sales'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print("Lower bound:", lower_bound)
print("Upper bound:", upper_bound)


Lower bound: 81.40750000000003
Upper bound: 911.9475


In [10]:
# Filter out rows outside the bounds
df_filtered = df_clean[(df_clean['sales'] >= lower_bound) & (df_clean['sales'] <= upper_bound)]

print("Rows before filtering:", len(df_clean))
print("Rows after filtering:", len(df_filtered))

# Preview final cleaned data
df_filtered.head()

Rows before filtering: 9410
Rows after filtering: 9223


,date,store_id,product_id,units_sold,unit_price,sales
0,2024-11-03,1001,262,23.0,29.55,679.65
1,2022-05-14,1005,256,20.0,25.32,506.40
2,2023-11-07,1008,245,25.0,31.58,789.50
3,2024-08-20,1000,232,19.0,25.91,492.29
4,2023-03-05,1001,255,28.0,22.45,628.60


### 🔍 Checkpoint:
- How many rows were removed as outliers?
- Do the remaining values seem more consistent?


## Step 5 (Optional): Group Sales by Month

If your data has a `'date'` column, we can extract the month and summarise sales.


In [11]:
# Convert to datetime if needed
df_filtered['date'] = pd.to_datetime(df_filtered['date'])

# Extract month number
df_filtered['Month'] = df_filtered['date'].dt.month

# Group and summarise sales
monthly_sales = df_filtered.groupby('Month')['sales'].sum()
monthly_sales

<ipython-input-11-8a0b37ca52a0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['date'] = pd.to_datetime(df_filtered['date'])
<ipython-input-11-8a0b37ca52a0>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Month'] = df_filtered['date'].dt.month


,sales
Month,
1,397589.77
2,338824.17
3,382800.10
4,398741.10
5,384460.97
6,357337.15
7,397114.00
8,406638.18
9,373164.15


## Reflection

- What did you learn about identifying and cleaning data?
- Was there a step that confused you at first? How did you resolve it?
- How would you explain the IQR method to a classmate?
